In [1]:
 from __future__ import print_function

import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from bs4 import BeautifulSoup
import base64
import re
import pandas as pd

In [2]:
# чтобы это запустилось, нужно сделать что написано здесь https://developers.google.com/gmail/api/quickstart/python
# часть кода взята отсюда https://www.geeksforgeeks.org/how-to-read-emails-from-gmail-using-gmail-api-in-python/

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

creds = None
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

service = build('gmail', 'v1', credentials=creds)
  
result = service.users().messages().list(userId='me').execute()

messages = result.get('messages')

In [3]:
for msg in messages:
    txt = service.users().messages().get(userId='me', id=msg['id']).execute()
    if 'STARRED' in txt['labelIds']:
        txt = service.users().messages().get(userId='me', id=msg['id']).execute()
        payload = txt['payload']
        headers = payload['headers']
        
        for d in headers:
            if d['name'] == 'Subject':
                subject = d['value']
            if d['name'] == 'From':
                sender = d['value']
        
        parts = payload.get('parts')[0]
        if 'АТОН' in subject:
            data = parts.get('parts')[0]['body']['data']
        else:
            data = parts['body']['data']
        data = data.replace("-","+").replace("_","/")
        decoded_data = base64.b64decode(data)
        soup = BeautifulSoup(decoded_data , "lxml")
        body = soup.body()
        
        lines = [re.sub('\s+', ' ', i.get_text(strip=True)) for i in soup.find_all('span')]
        lines = [i for i in lines if len(i) > 20]
        lines.extend(['конец аналитической записки', ''])
        with open('записки.txt', 'a') as f:
            for line in lines:
                try:
                    f.write(line)
                    f.write('\n')
                except:
                    pass

In [143]:
# для теста
lines = [re.sub('\s+', ' ', i.get_text(strip=True)) for i in soup.find_all('span')]
lines = [i for i in lines if len(i) > 20]
lines.extend(['конец аналитической записки', ''])
with open('news1.txt', 'a') as f:
    for line in lines:
        try:
            f.write(line)
            f.write('\n')
        except:
            pass